In [1]:
import sys
sys.path.append('../personal_pkgs/')
import pickle
import pandas as pd
import oop_func as func
import personal_pkg as ref
import parsing_func as pars
import os
import re
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
from IPython.display import Markdown

In [2]:
fm_base_df = pd.read_csv('fm_base_df.csv')
f_base_df = pd.read_csv('f_base_df.csv')
sc_base_df = pd.read_csv('sc_base_df.csv')
dr_base_df = pd.read_csv('dr_base_df.csv')
base_df = pd.read_csv('base_df.csv')

In [3]:
testing_df = base_df[base_df['journal'].notnull()]
testing_df.isnull().sum()

author      50
from         0
index        0
journal      0
paper        0
year       295
dtype: int64

- 2002-2008 : AJFS , 증권학회지 , 한국증권학회지 -> 증권학회지
- 2009 - : AJFS , 증권학회

In [4]:
def tuning_the_journal(base_df) :
    testing_df = base_df[base_df['journal'].notnull()]
    journal_ls = [i for i in testing_df['journal'].tolist()]
    journal_ls1 = [i.replace('”','') for i in journal_ls]
    journal_ls2 = [re.sub('제\s*[0-9]{,5}\s*권|제\s*[0-9]{,5}\s*호|제\s*[0-9]{,5}\s*집','',i) for i in journal_ls1]
    journal_ls3 = [','.join(re.findall('[a-zA-Z가-힣\s]',i)).replace(",",'').strip().lower() for i in journal_ls2]
    journal_ls4 = [re.sub('vol$|no$|vol no','',i).strip() for i in journal_ls3]
    testing_df['journal'] = journal_ls4
    testing_df['year'] = testing_df['year'].apply(lambda x : str(x).split('.')[0])
    return testing_df.loc[:,['author','journal','year','paper','from']]

In [5]:
fm_base_df = tuning_the_journal(fm_base_df)
f_base_df = tuning_the_journal(f_base_df)
sc_base_df = tuning_the_journal(sc_base_df)
dr_base_df = tuning_the_journal(dr_base_df)
base_df = tuning_the_journal(base_df)

In [6]:
def adjust_the_journal_under_2009(base_df) : 
    base_df.reset_index(drop=True,inplace=True)
    testing_df = base_df.copy()

    for idx in range(len(testing_df)) :
        if ('asiapacific journal of financial studies' in testing_df['journal'][idx]) or \
        ('한국증권학회지' in testing_df['journal'][idx]) or ('증권학회지' in testing_df['journal'][idx]) : 
            try : 
                if int(testing_df['year'][idx]) < 2009 : 
                    testing_df['journal'][idx] = '증권학회지'
            except : testing_df['journal'][idx] = '증권학회지'


    for idx in range(len(testing_df)) :
        if ('한국증권학회지' in testing_df['journal'][idx]) or ('증권학회지' in testing_df['journal'][idx]) : 
            try : 
                if int(testing_df['year'][idx]) > 2009 : 
                    testing_df['journal'][idx] = '증권학회지'
            except : pass # year 가 nan 인 데이터는 검색결과 2009년 이전의 논문이였다.
    return testing_df

In [7]:
new_base_df = adjust_the_journal_under_2009(base_df)
new_fm_base_df = adjust_the_journal_under_2009(fm_base_df)
new_f_base_df = adjust_the_journal_under_2009(f_base_df)
new_sc_base_df = adjust_the_journal_under_2009(sc_base_df)
new_dr_base_df = adjust_the_journal_under_2009(dr_base_df)

In [9]:
def make_counter_df_under_journal(base_df) :
    df1 = pd.DataFrame.from_dict(Counter(base_df['journal'].tolist()),orient='index',columns=['count'])
    df2 = df1.sort_values('count',ascending=False)
    return df2

In [13]:
journal_counter_df = make_counter_df_under_journal(new_base_df)
journal_counter_df.reset_index(inplace=True)
journal_counter_df.to_csv('journal_counter_df.csv',index=False)

In [11]:
display(Markdown('### 한국재무관리학회'))
display(make_counter_df_under_journal(new_fm_base_df).head(10))
display(Markdown('### 한국재무학회'))
display(make_counter_df_under_journal(new_f_base_df).head(10))
display(Markdown('### 한국증권학회지'))
display(make_counter_df_under_journal(new_sc_base_df).head(10))
display(Markdown('### 한국파생상품학회'))
display(make_counter_df_under_journal(new_dr_base_df).head(10))

### 한국재무관리학회

,count
journal of finance,1151
journal of financial economics,958
증권학회지,404
재무관리연구,398
review of financial studies,318
재무연구,257
journal of financial and quantitative analysis,199
american economic review,155
econometrica,153
journal of banking and finance,148


### 한국재무학회

,count
journal of finance,527
journal of financial economics,411
증권학회지,204
재무연구,179
journal of finance vol,144
재무관리연구,132
journal of financial economics vol,128
review of financial studies,113
econometrica,74
journal of political economy,61


### 한국증권학회지

,count
증권학회지,701
journal of financial economics,700
journal of finance,686
재무연구,313
재무관리연구,239
review of financial studies,157
journal of political economy,129
회계학연구,87
american economic review,85
선물연구,85


### 한국파생상품학회

,count
journal of finance,247
선물연구,164
journal of financial economics,133
review of financial studies,132
증권학회지,78
econometrica,63
journal of futures markets,53
journal of financial and quantitative analysis,52
재무관리연구,50
재무연구,47
